<a href="https://colab.research.google.com/github/omkarpat/BotBuilder-Location/blob/master/Baseline_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Colab settings/mount
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My\ Drive/CSE\ 240/Project/Data/spectrogram_images/

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/CSE 240/Project/Data/spectrogram_images


In [0]:
import os
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from keras import optimizers
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, SeparableConv2D, MaxPooling3D, Conv3D, DepthwiseConv2D
from keras.models import model_from_json

In [0]:
base_path = "."
concentration_data_array = None
relaxed_data_array = None
#relaxed_data_array = np.load("relaxed_array_updated_spectrograms_final.npy")
for filename in os.listdir(base_path):
  if "concentration_array_updated_spectrograms" in filename and "final" not in filename:
    print(filename)
    concentration_data_array = np.concatenate((concentration_data_array, np.load(filename))) if concentration_data_array is not None else np.load(filename)
  if "relaxed_array_updated_spectrograms" in filename:
    print(filename)
    relaxed_data_array = np.concatenate((relaxed_data_array, np.load(filename))) if relaxed_data_array is not None else np.load(filename)

concentration_array_updated_spectrograms_1.npy
concentration_array_updated_spectrograms_2.npy
concentration_array_updated_spectrograms_3.npy
concentration_array_updated_spectrograms_4.npy
concentration_array_updated_spectrograms_5.npy
concentration_array_updated_spectrograms_6.npy
relaxed_array_updated_spectrograms_1.npy
relaxed_array_updated_spectrograms_2.npy
relaxed_array_updated_spectrograms_3.npy
relaxed_array_updated_spectrograms_4.npy
relaxed_array_updated_spectrograms_5.npy
relaxed_array_updated_spectrograms_final.npy


In [0]:
print(concentration_data_array.shape, relaxed_data_array.shape)

(2160, 29, 43, 42) (2124, 29, 43, 42)


In [0]:
X = np.concatenate((relaxed_data_array, concentration_data_array))
relaxed_data_array, concentration_data_array = None, None

In [0]:
Y = np.concatenate((np.zeros((2160,1)),np.ones((2124,1))))
print(X.shape, Y.shape)

(4284, 29, 43, 42) (4284, 1)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X, Y = None, None
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.15, random_state=42)
print(X_train.shape, X_test.shape, X_valid.shape, y_train.shape, y_test.shape, y_valid.shape)

(3094, 29, 43, 42) (643, 29, 43, 42) (547, 29, 43, 42) (3094, 1) (643, 1) (547, 1)


In [0]:
model2 = Sequential()
model2.add(Conv2D(64, kernel_size=5, strides=(1,1), padding='valid', activation='relu', input_shape=(29, 43, 42)))
model2.add(Conv2D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Conv2D(64, kernel_size=5, strides=(1,1), padding='valid', activation='relu'))
model2.add(Conv2D(128, kernel_size=3, activation='relu'))
model2.add(MaxPooling2D(pool_size=(2, 2)))
model2.add(Flatten())
model2.add(Dense(100, activation='relu'))
model2.add(Dense(50, activation='relu'))
model2.add(Dense(1, activation='sigmoid'))
sgd = optimizers.SGD(lr=0.000001, momentum=0.7, decay=1e-6)
model2.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])
# checkpoint
filepath="model2-weights-improvement-{epoch:02d}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model2.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=50, batch_size=32, callbacks=callbacks_list, shuffle=True)

Train on 3094 samples, validate on 547 samples
Epoch 1/50
3094/3094 [==============================] - 69s 22ms/step - loss: 2.7152 - acc: 0.5068 - val_loss: 2.1801 - val_acc: 0.5119

Epoch 00001: acc improved from -inf to 0.50679, saving model to model2-weights-improvement-01.hdf5
Epoch 2/50
3094/3094 [==============================] - 69s 22ms/step - loss: 2.1169 - acc: 0.5019 - val_loss: 1.8819 - val_acc: 0.5484

Epoch 00002: acc did not improve from 0.50679
Epoch 3/50
3094/3094 [==============================] - 68s 22ms/step - loss: 1.8527 - acc: 0.5233 - val_loss: 1.7316 - val_acc: 0.5558

Epoch 00003: acc improved from 0.50679 to 0.52327, saving model to model2-weights-improvement-03.hdf5
Epoch 4/50
2272/3094 [=====================>........] - ETA: 17s - loss: 1.7815 - acc: 0.5312

In [46]:
metrics = model2.evaluate(X_test,y_test)
print(metrics)

643/643 [==============================] - 4s 6ms/step
[3.35662171462461, 0.5365474339962747]


In [49]:
model2_json = model2.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model2_json)
# serialize weights to HDF5
model2.save_weights("model2.h5")
print("Saved model 2 to disk")

Saved model 2 to disk


In [52]:
# load json and create model
json_file = open('model2.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model2.h5")
print("Loaded model 2 from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X_test,y_test, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

Loaded model 2 from disk
acc: 53.65%
